### viewers setup

In [1]:
import os
from os.path import join as joinp
from brainbuilder.utils import viewer
from IPython.display import HTML
#location where output will be stored under viewer directory:
directory = "out"

# the output directory should be in the viewer directory to be accessible by the viewer.
output_directory = joinp("..", "viewer", directory)

if not os.path.isdir(output_directory):
    os.makedirs(output_directory)
    
#change display_server to the one that serves the viewer. 
display_server = "http://localhost:8080/"
display_path = display_server + '#' + directory + '/'

iframe = '<iframe src="'+ display_path + '%s"  scrolling="no" width=700 height=350></iframe>'

### loaders initialization

In [2]:
from brainbuilder.utils import genbrain as gb
from brainbuilder.utils import bbp
from brainbuilder.utils import traits as tt

# Brain Builder modules

##   Build.Region: Region of Interest

In [3]:
from brainbuilder.select_region import select_region, select_hemisphere

### input parameters

In [4]:
annotation = gb.MetaIO.load("../data/P56_Mouse_annotation/annotation.mhd")
hierarchy = gb.load_hierarchy("../data/P56_Mouse_annotation/annotation_hierarchy.json")['msg'][0]
full_density = gb.MetaIO.load("../data/atlasVolume/atlasVolume.mhd")
region_name = "Primary somatosensory area"

### run module

In [5]:
density_raw = select_region(annotation.raw, full_density.raw, hierarchy, region_name)
density_raw = select_hemisphere(density_raw)

### output

In [6]:
import copy
selected_region_mhd = copy.copy(full_density.mhd);
selected_region_mhd['ElementDataFile'] = 'selected_region.raw'
gb.save_meta_io(joinp(output_directory,'selected_region.mhd'),selected_region_mhd, joinp(output_directory,'selected_region.raw'), density_raw)
HTML(iframe % "selected_region.mhd")

##  Build.Cells:  Cell Positions

In [7]:
from brainbuilder.cell_positioning import cell_positioning

### input parameters

In [8]:
voxel_dimensions = full_density.mhd['ElementSpacing']
total_cell_count = 200000

### run module

In [9]:
positions = cell_positioning(density_raw, voxel_dimensions, total_cell_count)

### output

In [10]:
viewer.serialize_points(joinp(output_directory,'positions.pts'), positions, [[50, 50, 50]]*len(positions))
HTML(iframe % "positions.pts")

## Build.EI:  E-I ratios

In [11]:
from brainbuilder.assignment_synapse_class import assign_synapse_class_from_spatial_dist

### input parameters

In [12]:
recipe_filename = "../data/bbp_recipe/builderRecipeAllPathways.xml"
recipe_sdist = bbp.load_recipe_as_spatial_distribution(recipe_filename,annotation.raw, hierarchy, region_name)
sclass_sdist = tt.reduce_distribution_collection(recipe_sdist, 'sClass')

### run module

In [13]:
chosen_synapse_class = assign_synapse_class_from_spatial_dist(positions, sclass_sdist, voxel_dimensions)

### output

In [14]:
viewer.export_points(joinp(output_directory, 'sclass.pts'),positions, chosen_synapse_class, 'sClass')
HTML(iframe % "sclass.pts")

## Build.Composition.ME: METype for Soma

In [15]:
from brainbuilder.assignment_metype import assign_metype

### run module

In [16]:
chosen_me = assign_metype(positions, chosen_synapse_class, recipe_sdist, voxel_dimensions)

### output

#### mtypes

In [17]:
viewer.export_points(joinp(output_directory, 'mtype.pts'), positions, [me[0] for me in chosen_me], 'mtype')
HTML(iframe % 'mtype.pts')

#### etypes

In [18]:
viewer.export_points(joinp(output_directory, 'etype.pts'), positions, [me[1] for me in chosen_me], 'etype')
HTML(iframe % 'etype.pts')

## Build.Placement: Morphology assignment

In [19]:
from brainbuilder.assignment_morphology import assign_morphology

### input parameters

In [20]:
neurondb_filename = "../data/bbp_recipe/neurondb.dat"
neuron_sdist = bbp.load_neurondb_v4_as_spatial_distribution(neurondb_filename, annotation.raw,hierarchy, region_name,percentile=0.92)

### run module

In [21]:
chosen_morphology = assign_morphology(positions, chosen_me, neuron_sdist, voxel_dimensions)

In [22]:
viewer.export_points(joinp(output_directory, 'morph.pts'), positions, chosen_morphology, 'morphology')
HTML(iframe % 'morph.pts')